# Finetune ruGPT3Small on essays

## Install enviroment

In [1]:
!pip3 install urllib3==1.25.4

     |████████████████████████████████| 133kB 22.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
!pip3 install transformers==2.8.0

     |████████████████████████████████| 573kB 13.1MB/s 
     |████████████████████████████████| 1.1MB 47.9MB/s 
     |████████████████████████████████| 3.7MB 53.3MB/s 
     |████████████████████████████████| 890kB 52.5MB/s 
     |████████████████████████████████| 133kB 55.3MB/s 
     |████████████████████████████████| 7.1MB 47.2MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=997f02c834cae77df6a96a06ab541a1ead9296f4d62233b9c2de669717c927d9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

--2020-12-19 22:03:43--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34150 (33K) [text/plain]
Saving to: ‘pretrain_transformers.py’

pretrain_transforme 100%[===================>]  33.35K  --.-KB/s    in 0s      

2020-12-19 22:03:43 (140 MB/s) - ‘pretrain_transformers.py’ saved [34150/34150]



In [4]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

--2020-12-19 22:03:47--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10474 (10K) [text/plain]
Saving to: ‘generate_transformers.py’

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

2020-12-19 22:03:47 (128 MB/s) - ‘generate_transformers.py’ saved [10474/10474]



In [5]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [6]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 7872 (delta 10), reused 7 (delta 0), pack-reused 7835
Receiving objects: 100% (7872/7872), 13.99 MiB | 26.58 MiB/s, done.
Resolving deltas: 100% (5357/5357), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-ugn__lzb
Created temporary directory: /tmp/pip-req-tracker-qcxtv2pt
Created requirements tracker '/tmp/pip-req-tracker-qcxtv2pt'
Created temporary directory: /tmp/pip-install-gcvahr66
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-_nddd507
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-qcxtv2pt'
    Running setup.py (path:/

## Add data to colab
Add essays file from google dirve:
* Add [file](https://drive.google.com/file/d/10ZsjTeaoihYA80n1G40O5YZmaGw0yOXk/view?usp=sharing) to your own drive
* And mount drive to colab

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = "drive/My Drive/essays.txt"
!ls "$data_path"

'drive/My Drive/essays.txt'


## Prepare data

In [ ]:
with open(data_path, "r") as file:
    text = file.read()

In [ ]:
valid_size = 5

In [ ]:
topics = []
all_essays = []
for line in text.split("</s>"):
    if "Тема:" in line and "Сочинение:" in line:
        essay_text = line.split("Сочинение:")
        if len(essay_text) == 2:
            topic = essay_text[0].replace("<s>", " ").replace("</s>", " ").strip()
            essay_text = essay_text[1].replace("<s>", " ").replace("</s>", " ").strip()
            essay_text = f"Сочинение: {essay_text}"
            essay_res = f"<s>{topic}\n{essay_text}</s>"
            all_essays.append(essay_res)
            topics.append(topic)

In [ ]:
import numpy as np
import random

In [ ]:
random.seed(1234)
np.random.seed(1234)

In [ ]:
unique_topics = list(set(topics))

In [ ]:
valid_topics = []

In [ ]:
for _ in range(valid_size):
    # Use randint for more speed (on big lists it is faster)
    idx = np.random.randint(0, len(unique_topics))
    valid_topics.append(unique_topics[idx])

In [ ]:
import nltk


train = []
valid = []
for topic, essay in zip(topics, all_essays):
    is_train = True
    for valid_topic in valid_topics:
        if (
            nltk.edit_distance(valid_topic, topic[:len(valid_topic)]) < 20 or
            nltk.edit_distance(valid_topic[:len(topic)], topic) < 20 or
            nltk.edit_distance(valid_topic[len(topic):], topic) < 20 or
            nltk.edit_distance(valid_topic, topic[len(valid_topic):]) < 20
            ):
            is_train = False
    if is_train:
        train.append(essay)
    else:
        valid.append(essay)

In [ ]:
len(train), len(valid)

(313, 87)

In [ ]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [ ]:
!python pretrain_transformers.py \
    --output_dir=essays_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --eval_data_file=val.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

2020-12-19 23:14:51.629030: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
12/19/2020 23:14:54 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
12/19/2020 23:14:54 - INFO - filelock -   Lock 140204873948072 acquired on /root/.cache/torch/transformers/767f65149d3a8095b8ac0370c3b2bbc7f05d863ff413b010d2f5206de0349fd0.1ede3f500f8b09312434582244c28d34f008436c5a38bb3afa159d2b38f8e758.lock
12/19/2020 23:14:54 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/sberbank-ai/rugpt3small_based_on_gpt2/config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp0zcoznhx
Downloading: 100% 608/608 [00:00<00:00, 726kB/s]
12/19/2020 23:14:54 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/sberbank-ai/rugpt3small_based_on_gpt2/c

## Check our model

In [ ]:
## Select topic
for idx in range(len(valid)):
  if "образует его общество" in valid[idx]:
    break

In [ ]:
valid[idx]

'<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: Эссе №1Человек это высшая ступень развития живых организмов на земле, субъект общественно-исторической деятельности и культуры, но важнейшей его характеристикой является биосоциальная сущность.Белинский В.Г. в своем выражении очень точно и ёмко охарактеризован двойственную природу человека. Во-первых, человек есть порождение природы, является цепью эволюции, таким же организмом, как и всё, что нас окружает. По биологическим признакам человек ничем не отличается от животного. Во-вторых, он есть порождение общества. Этот момент более сложен. Ясно только то, что благодаря общественному развитию человек стал человеком.Человек без общества ничто, недаром в древности изгнание из общества являлось самим страшным наказанием. Очень много в современном мире примеров синдрома«маугли», когда воспитание ребенок получил от животного и поэтому ведет себя как он, а не как человек, что говорит лишь 

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=essays_model \
    --k=5 \
    --p=0.95 \
    --length=500

2020-10-28 15:45:33.604619: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   Model name 'essays_model' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'essays_model' is a path, a model identifier, or url to a directory containing tokenizer files.
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   Didn't find file essays_model/added_tokens.json. We won't load it.
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   loading file essays_model/vocab.json
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   loading file essays_model/merges.txt
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   loading file None
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   loading file essays_model/special_tokens_map.json
10/28/2020 15:45:35 - INFO - tra